<a href="https://colab.research.google.com/github/Hann1bal/DecisionSupportSystemCourse/blob/main/TFRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -q tensorflow-recommenders


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement StringIO (from versions: none)
ERROR: No matching distribution found for StringIO


In [ ]:
import os 
import tensorflow as tf
import requests
from zipfile import ZipFile
from io import BytesIO
url = "https://files.grouplens.org/datasets/movielens/ml-20m.zip"
file = requests.get(url).content
#zip = 
zip =  open("./20ml.zip", "wb")
zip.write(file)
zip.close()

zf = ZipFile("./20ml.zip")
zf.extractall("./movie/dataset/")
zf.close()

In [ ]:
!ls ./movie/dataset/ml-20m


genome-scores.csv  links.csv   ratings.csv  tags.csv
genome-tags.csv    movies.csv  README.txt


In [ ]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds

In [ ]:
import random
# rating_dataset = tf.data.experimental.make_csv_dataset("./movie/dataset/ml-20m/ratings.csv", 
#                                                 batch_size = 35,
#                                                 num_epochs=1,
#                                                 shuffle = True,
#                                                 )
# movie_dataset = tf.data.experimental.make_csv_dataset("./movie/dataset/ml-20m/movies.csv", 
#                                                 batch_size = 35,
#                                                 num_epochs=1,
#                                                 shuffle = True,
#                                                 )
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [ ]:
ratings = ratings.map(lambda x:{
    "movie_title":x["movie_title"],
    "user_id":x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

In [ ]:
# ratings = rating_dataset.map(lambda x:{
#     "movieId":x["movieId"],
#     "userId":x["userId"]
# })
# movies = movie_dataset.map(lambda x: x["movieId"])

In [ ]:
tf.random.set_seed(55)
shuffled = ratings.shuffle(100_000, seed = 55, reshuffle_each_iteration=False)
train  = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [ ]:
movie_titles = movies.batch(1_000)
user_ids= rating.batch(1000000).map(lambda x: x["user_id"])

uniq_user_id = np.unique(np.concatenate(list(user_ids)))
uniq_movie_id = np.unique(np.concatenate(list(movie_titles)))

In [ ]:

user_model = tf.keras.Sequential([tf.keras.layers.StringLookup(vocabulary=uniq_user_id, mask_token=None),
                                  tf.keras.layers.Embedding(len(uniq_user_id)+1, 32)])

In [ ]:


movie_model = tf.keras.Sequential([tf.keras.layers.StringLookup(vocabulary=uniq_movie_id, mask_token=None),
                                  tf.keras.layers.Embedding(len(uniq_movie_id)+1, 32)])

In [ ]:
metrics = tfrs.metrics.FactorizedTopK(candidates= movies.batch(256).map(movie_model))

In [ ]:
task = tfrs.tasks.Retrieval(metrics=metrics)

In [ ]:
class MovieMdoel(tfrs.Model):
  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model = movie_model
    self.user_model = user_model
    self.task = task
  def compute_loss( self, features, training = False):
    user_embeddings = self.user_model(features["user_id"])
    positive_movie_embeddings = self.movie_model(features["movie_title"])
    return self.task(user_embeddings, positive_movie_embeddings)
    

In [ ]:
model = MovieMdoel(user_model, movie_model)
model.compile(optimizer = tf.keras.optimizers.Adagrad(learning_rate = 0.1))

In [ ]:
cache_data_train = train.shuffle(100000).batch(8092).cache()
cache_data_test = test.batch(4096).cache()

In [ ]:
model.fit(cache_data_train, epochs=15)

Epoch 1/15
10/10 [==============================] - 27s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0013 - factorized_top_k/top_5_categorical_accuracy: 0.0102 - factorized_top_k/top_10_categorical_accuracy: 0.0204 - factorized_top_k/top_50_categorical_accuracy: 0.1009 - factorized_top_k/top_100_categorical_accuracy: 0.1759 - loss: 70610.9808 - regularization_loss: 0.0000e+00 - total_loss: 70610.9808
Epoch 2/15
10/10 [==============================] - 20s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0026 - factorized_top_k/top_5_categorical_accuracy: 0.0180 - factorized_top_k/top_10_categorical_accuracy: 0.0374 - factorized_top_k/top_50_categorical_accuracy: 0.1644 - factorized_top_k/top_100_categorical_accuracy: 0.2886 - loss: 68286.5220 - regularization_loss: 0.0000e+00 - total_loss: 68286.5220
Epoch 3/15
10/10 [==============================] - 19s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0035 - factorized_top_k/top_5_categorical_accuracy: 0.0

In [ ]:
model.evaluate(cache_data_test, return_dict=True)

5/5 [==============================] - 7s 716ms/step - factorized_top_k/top_1_categorical_accuracy: 2.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0021 - factorized_top_k/top_10_categorical_accuracy: 0.0066 - factorized_top_k/top_50_categorical_accuracy: 0.0738 - factorized_top_k/top_100_categorical_accuracy: 0.1679 - loss: 32050.2363 - regularization_loss: 0.0000e+00 - total_loss: 32050.2363


{'factorized_top_k/top_1_categorical_accuracy': 0.00019999999494757503,
 'factorized_top_k/top_5_categorical_accuracy': 0.002099999925121665,
 'factorized_top_k/top_10_categorical_accuracy': 0.006649999879300594,
 'factorized_top_k/top_50_categorical_accuracy': 0.07384999841451645,
 'factorized_top_k/top_100_categorical_accuracy': 0.1678999960422516,
 'loss': 29129.140625,
 'regularization_loss': 0,
 'total_loss': 29129.140625}

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)
for i in  index(tf.constant(["93"])):
  print(f"{i}")

[[6.609688  6.16212   6.08993   6.0479083 6.0459013 6.032731  5.927784
  5.9244375 5.923994  5.8971434]]
[[b'Harriet the Spy (1996)' b'First Wives Club, The (1996)'
  b'Birdcage, The (1996)' b'Zeus and Roxanne (1997)'
  b'In Love and War (1996)' b'Very Brady Sequel, A (1996)'
  b'Out to Sea (1997)' b'One Fine Day (1996)' b'Evening Star, The (1996)'
  b'Truth About Cats & Dogs, The (1996)']]


In [ ]:
with tempfile.TemporaryDirectory() as tmp:
  path = os.path.join(tmp, "model")
  tf.saved_model.save(index, path)
  loaded_model = tf.saved_model.load(path)
  scores, titels = loaded_model(["90"])